In [1]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'
# a_pattern = 'SI06122.Chn'

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
# import plotly.graph_objects as go

In [3]:
from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [4]:
ogra = Ograyspy()
print(ogra.info_node)
# to_be_found = 'Genie_Transfer'
to_be_found = 'some_spectra'
print('\nExec ogra.define_files_folder(to_be_found):')
ogra.define_files_folder(to_be_found)
print(ogra.pkl_folder_files)
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
del ogra

Found folder name:  /home/jovyan/.bashrc
Parts:  ('/', 'home', 'jovyan', '.bashrc')
No. spec files:  0
jupyter-terrabela-2dograyspy-2dzkasrbmk

Exec ogra.define_files_folder(to_be_found):
Found folder name:  /home/jovyan/data/some_spectra
Parts:  ('/', 'home', 'jovyan', 'data', 'some_spectra')
No. spec files:  11
Linuxjupyter-terrabela-2dograyspy-2dzkasrbmk.pkl


In [5]:
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Linux-5.15.0-79-generic-x86_64-with-glibc2.27,x86_64,Linux,jupyter-terrabela-2dograyspy-2dzkasrbmk,/home/jovyan,/home/jovyan/data/some_spectra,11,0,,,"[/home/jovyan/data/some_spectra/CCI0902-I.Chn,...","[CCI0902-I.Chn, CCI0202-I.Chn, CCI0903-I.Chn, ...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",Linuxjupyter-terrabela-2dograyspy-2dzkasrbmk.pkl


In [6]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [7]:
reduc_nms

['CCI0902-I.Chn',
 'CCI0202-I.Chn',
 'CCI0903-I.Chn',
 'CCI0501-I.Chn',
 'CCI2302-I.Chn',
 'CCI1201-I.Chn',
 'CCI1503-F.Chn',
 'CCI1901-I.Chn',
 'CCI1602-I.Chn',
 'CCI2601-I.Chn',
 'CCI1603-I.Chn']

In [8]:
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06122.Chn'

In [9]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Final choices:
spectra_path: /home/jovyan/data/some_spectra


UnboundLocalError: local variable 'a_spec_name' referenced before assignment

In [ ]:
nucl_iear1_df = pd.read_pickle('nucl_iear1_list.pkl')
nucl_iear1_df

In [ ]:
# 2023-Jun-15: Setting gamma lines/ranges to dismiss in the analysis
df1 = nucl_iear1_df
df1["is_to_consider"] = True
df1.loc[((df1.energy > 509) & (df1.energy < 513)) | (df1.energy < 100), "is_to_consider"] = False
del df1
nucl_iear1_df

In [ ]:
nucl_iear1_df

In [ ]:
a_spec = Spec(f_name, reduced_f_name)

In [ ]:
dir(a_spec)

In [ ]:
orig_ser = a_spec.origin_spec_ser_an
vars(orig_ser)

In [ ]:
a_spec.total_analysis(gener_dataframe=True)
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

In [ ]:
vars(orig_ser)

In [ ]:
net_ser = a_spec.net_spec_ser_an
vars(net_ser)

In [ ]:
print(a_spec.pkl_file)

In [ ]:
a_spec.pkl_file

In [ ]:
# Só para verificar se o pkl_file pode ser recuperado:
read_analysis_spec = pd.read_pickle(a_spec.pkl_file)

In [ ]:
read_orig = read_analysis_spec['origin_spec_ser_an'][0]
orig_ser = read_orig
vars(orig_ser)

In [ ]:
read_net = read_analysis_spec['net_spec_ser_an'][0]
net_ser = read_net
vars(net_ser)

In [ ]:
# 2023-May-26
# Ok! Espectro gravado, depois lido, então vamos prosseguir
# com a identificação dos nuclídeos.
# Agora, posso ler e analisar um espectro, gravá-lo e depois, em outro momento,
# ler o pkl com a análise.

In [ ]:
x_nz = orig_ser.chans_nzero
y_nz = orig_ser.counts_nzero

In [ ]:
chans = net_ser.x_s
ys_net_counts = net_ser.y_s
peaks_net = net_ser.pk_parms.peaks
peaks_orig = orig_ser.pk_parms.peaks
counts = orig_ser.y_s
chans_nzero = orig_ser.chans_nzero
counts_nzero = orig_ser.counts_nzero
unc_y = orig_ser.unc_y
eval_y = orig_ser.y_smoothed
eval_bl = orig_ser.eval_baseline
fin_bl = orig_ser.final_baseline
inis = orig_ser.pk_parms.propts['left_bases']
fins = orig_ser.pk_parms.propts['right_bases']

In [ ]:
# 2023-Jun-15
# CRUCIAL step: take the dict net_ser.pk_parms' keys/values and organize them as a pd.Dataframe
vars_pkprms = vars(net_ser.pk_parms)
keys_to_get = ['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances']
prep_for_dict = [(key, vars_pkprms[key]) for key in keys_to_get]
pks_dict = dict(prep_for_dict)
peaks_df = pd.DataFrame.from_dict(pks_dict)
peaks_df

In [ ]:
pr_pk = dict([('pk_hei',net_ser.pk_parms.propts['peak_heights']),
              ('lf_thr',net_ser.pk_parms.propts['left_thresholds']),
              ('rg_thr',net_ser.pk_parms.propts['right_thresholds']),
              ('promns',net_ser.pk_parms.propts['prominences']),
              ('lf_bas',net_ser.pk_parms.propts['left_bases']),
              ('rg_bas',net_ser.pk_parms.propts['right_bases']),
              ('widths',net_ser.pk_parms.propts['widths']),
              ('wi_hei',net_ser.pk_parms.propts['width_heights']),
              ('lf_ips',net_ser.pk_parms.propts['left_ips']),
              ('rg_ips',net_ser.pk_parms.propts['right_ips'])])

In [ ]:
pks_properties_df = pd.DataFrame(pr_pk)
pks_properties_df

In [ ]:
pks_comprehensive_df = pd.concat([peaks_df, pks_properties_df],axis=1)
pks_comprehensive_df

In [ ]:
def add_engy_to_pks_df(peaks_net_kev_df, func_en):
    peaks_net_kev_df['engy_pk_det'] = func_en(peaks_net_kev_df.centroids)

In [ ]:
add_engy_to_pks_df(pks_comprehensive_df, a_spec.channel_energy_calib.get_energy)

In [ ]:
pks_comprehensive_df

In [ ]:
vars()

In [ ]:
nucl_iear1_df

In [ ]:
nucl_iear1_selctd_gamms_df = nucl_iear1_df.loc [
    (nucl_iear1_df.intensity > 1.0) & nucl_iear1_df.is_to_consider
]
nucl_iear1_selctd_gamms_df

In [ ]:
cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
cross_df

In [ ]:
def create_matching_peaks_df(pks_df, en_toler, must_be_key_gamma=False):
    if must_be_key_gamma:
        aux_df = pd.DataFrame(pks_df.loc[pks_df.is_key_gamma])
    else:
        aux_df = pd.DataFrame(pks_df)
    return aux_df.loc[np.abs(pks_df.delta_en) < en_toler] 

In [ ]:
en_toler_calib = 3.0

In [ ]:
matching_peaks_df = create_matching_peaks_df(cross_df, en_toler_calib, must_be_key_gamma=True)
matching_peaks_df

In [ ]:
matching_peaks_df = create_matching_peaks_df(cross_df, en_toler_calib, must_be_key_gamma=False)
matching_peaks_df

In [ ]:
X_energy = np.array(matching_peaks_df.energy).reshape(-1,1)
y_delta_en = np.array(matching_peaks_df.delta_en)
# Robustly fit linear model with RANSAC algorithm
ransac = linear_model.RANSACRegressor()
ransac.fit(X_energy, y_delta_en)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)
matching_peaks_df["recalib_engy_ransac"] = matching_peaks_df.engy_pk_det - ransac.predict(X_energy)
# cross_df["recalib_engy_ransac"] = cross_df.engy_pk_det - ransac.predict(X_energy)

In [ ]:
# 2023-Aug-18 PAREI AQUI: Ver como eliminar as duplicadas (tipo 238 e 241 keV, etc)
matching_peaks_df

In [ ]:
xzdfgxfrt jm dyjd

In [ ]:
2+3

In [ ]:
cross_df_2 = pd.merge(df1, nucl_lib_df, how='cross', suffixes=('_x', None))
cross_df_2.drop(cross_df_2.filter(regex='_x$').columns, axis=1, inplace=True)
cross_df_2["new_delta_en"] = cross_df_2.recalib_engy_ransac - cross_df_2.energy

In [ ]:
df1

In [ ]:
def spec_engy_recalib(peaks_net_kev_df, nucl_lib_df, func_en,
                      en_toler_calib=3.0, ):
    # Library-based spectrum energy/channel recalibration
    # TODO: implement option to update self.channel_energy_calib

    """Perform a library-based spectrum energy/channel recalibration
    :param func_en: energy function
    :type callable
    :param peaks_net_kev_df: single col 'energy_detected' w/ peaks' energies detected
    :type peaks_net_kev_df: pd.Dataframe
    :param nucl_lib_df: nuclide library based on IAEA format as a Pandas dataframe
    :type nucl_lib_df: pd.Dataframe
    :param en_toler_calib: energy difference accetable to take a peak for automatic calibration
        (default is 3.0)
    :type en_toler_calib: float
    :returns: a pd.Dataframe with each row as an identified peak with parameters
    :rtype: pd.Dataframe
    """

    # peaks_net_kev_df['engy_pk_det'] = func_en(peaks_net_kev_df.centroids)
    # cross_df = pd.merge(peaks_net_kev_df, nucl_lib_df, how='cross')
    # cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
    # df1 = pd.DataFrame(cross_df.loc [(np.abs(cross_df.delta_en) < en_toler_calib)] )
    X_energy = np.array(df1.energy).reshape(-1,1)
    y_delta_en = np.array(df1.delta_en)
    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor()
    ransac.fit(X_energy, y_delta_en)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    df1["recalib_engy_ransac"] = df1.engy_pk_det - ransac.predict(X_energy)
    # cross_df["recalib_engy_ransac"] = cross_df.engy_pk_det - ransac.predict(X_energy)

    cross_df_2 = pd.merge(df1, nucl_lib_df, how='cross', suffixes=('_x', None))
    cross_df_2.drop(cross_df_2.filter(regex='_x$').columns, axis=1, inplace=True)
    cross_df_2["new_delta_en"] = cross_df_2.recalib_engy_ransac - cross_df_2.energy
    # return cross_df_2
    return df1
    
    # df_result = pd.DataFrame(cross_df_2.loc [(np.abs(cross_df_2.new_delta_en) < en_toler_ident)] )
    # return peaks_net_kev_df, nucl_lib_df, df1, cross_df_2, df_result
    # return cross_df_2

In [ ]:
def spec_engy_identif(cross_df_2, en_toler_ident=0.5):
    # Library-based spectrum energy/channel recalibration
    # TODO: implement options for activity calculation

    """Perform a library-based spectrum energy/channel recalibration
    :param nucl_lib_df: nuclide library based on IAEA format as a Pandas dataframe
    :type nucl_lib_df: pd.Dataframe
    :param en_toler_ident: energy difference accetable to take a detected peak as possibly identified
        (default is 0.5)
    :type en_toler_ident: float
    :returns: a pd.Dataframe with aggregated activity calculations
    :rtype: pd.Dataframe
    """

    df_result = pd.DataFrame(cross_df_2.loc [(np.abs(cross_df_2.new_delta_en) < en_toler_ident)] )
    return df_result
    # return cross_df_2

In [ ]:
chnm dgfynj dtfyj dtyj dy jd

In [ ]:
dir()

In [ ]:
engy_recalib_pks_df = spec_engy_recalib(big_df, nucl_iear1_selctd_gamms_df, a_spec.channel_energy_calib.get_energy)
engy_recalib_pks_df

In [ ]:
big_df

In [ ]:
d1, d2, d3, d4, d5 = spec_engy_identif(
    big_df, nucl_iear1_df, a_spec.channel_energy_calib.get_energy
)

In [ ]:
d1

In [ ]:
d2

In [ ]:
d3

In [ ]:
d4

In [ ]:
d5

In [ ]:
d5.columns

In [ ]:
# 2023-Jun-20: Cálculo das atividades

In [ ]:
# Ver exatamente o que são esses elementos de Polynomial

In [ ]:
# 2023-Jun-14
# Loading calibration pkl file:
calib_pkl_name = 'f100_gmx_2021.pkl'
calib_df = pd.read_pickle(calib_pkl_name)
calib_df

In [ ]:
p_eff, _ = calib_df.effic_func[3]
p_eff

In [ ]:
def en_eff(engy):
    return np.exp(p_eff(np.log(engy)))

In [ ]:
en_eff(1332)

In [ ]:
# 2023-Aug-18
# AQUI escolher nucludeo que tiver mais picos identificados (Ag-110m tem mais que Cs-137)
# Por enquanto, usar a calibracao robusta, que deve bastar.

In [ ]:
d5

In [ ]:
d5["disintegr"] = d5.rough_sums / (en_eff(d5.energy) * 1e-4 * d5.intensity)

In [ ]:
# 2023-Ago-02 Aparece a nova coluna "disintegr"
d5

In [ ]:
# d5_where = d5.where(d5.nuclide_name=="110ag")
# d5_where
d5_loc = d5.loc[(d5.nuclide_name=="110ag")]
d5_loc

In [ ]:
# 2023-Ago-4 PAREI AQUI. Fazer a indexação para o cálculo da atividade considerando a média das linhas de cada nuclídeo
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#dataframe-column-selection-in-groupby
d5_grouped = d5.groupby("nuclide_name")[['energy','intensity','centroids','rough_sums','variances','disintegr']]
# d5_grouped = d5.groupby("nuclide_name")[['disintegr']]
# sdfsdf = pd.DataFrame([[d5_grouped.mean(), d5_grouped.median()]]) 
# sdfsdf
d5_grouped.mean()

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#built-in-aggregation-methods
d5_aggr_df = pd.DataFrame(d5.groupby("nuclide_name")[['disintegr']].mean())
d5_aggr_df['dsnt_median'] = d5.groupby("nuclide_name")[['disintegr']].median()
d5_aggr_df['dsnt_counts'] = d5.groupby("nuclide_name")[['disintegr']].count()
d5_aggr_df['dsnt_sem'] = d5.groupby("nuclide_name")[['disintegr']].sem()
d5_aggr_df['dsnt_std'] = d5.groupby("nuclide_name")[['disintegr']].std()
d5_aggr_df

In [ ]:
d5.groupby("nuclide_name")[['disintegr']].describe()

In [ ]:
d5_aggregated['disint_mean'] = d5_grouped.mean()
d5_aggregated['disint_count'] = d5_grouped.count()
d5_aggregated

In [ ]:
d5

In [ ]:
a_spec.lv_time

In [ ]:
# Sample size (L, kg, g etc.)
samp_size = 0.1

In [ ]:
a_spec.sam_descr

In [ ]:
a_spec.source_datetime

In [ ]:
a_spec.start_datetime

In [ ]:
# 2023-May-24
# Implementing:
#  - nuclide labels
# https://plotly.com/python/text-and-annotations/
#  - and energy search in pandas
# https://stackoverflow.com/questions/63529555/get-list-of-occurrences-using-pandas

In [ ]:
# counts_nzero_cps = counts_nzero / a_spec.lv_time
# unc_y_cps = unc_y / a_spec.lv_time
# fin_bl_cps = fin_bl / a_spec.lv_time
# counts_cps = counts / a_spec.lv_time
# en_kev_nzero = get_energy(chans_nzero)
# en_kev = get_energy(chans)
# peaks_net_kev = get_energy(peaks_net)

In [ ]:
help(spec_engy_identif)

In [ ]:
# 2023-Jun-13
# PAREI AQUI
# Trazer o trabalho feito em
# radionuclide_analysis.ipynb e adaptar ao que está aqui
# especialmente na identificaćão dos nucl[ideos

# Depois, deletar radionuclide_analysis.ipynb.

In [ ]:
xdfgh rxfth rxthj rds

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)


frames = [df1, df2, df3]

result = pd.concat(frames)
result

In [ ]:
result = pd.concat(frames, keys=["x", "y", "z"])
result

In [ ]:
result.loc[result.A=='A1'].loc[result.C=='C0']

In [ ]:
# https://stackoverflow.com/questions/63529555

In [ ]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [ ]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

In [ ]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

In [ ]:
# The End